In [1]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
out_dir = "./data/out"
in_dir = "./data/in"

In [3]:
optimizer_name = "brute force"
set_names = ["set-1", "set-2", "set-3"]
optimizer_dir = os.path.join(out_dir, optimizer_name)

In [4]:
def ftos(num):
    out = "{:,.1f}".format(num).replace(',', ' ')
    return out.replace(".", ",")

In [5]:
def rtof(frac):
    num, denom = frac.split("/")
    return float(num)/float(denom)

In [6]:
def convert_level(level):
    return ftos((rtof(level)-1.)*100)

In [7]:
def list_to_string(arr):
    begin = r"\([\begin{array}{" + "c"*len(arr) + "} " 
    end = r"\end{array}]\)"
    out = begin
    
    for i in range(len(arr)-1):
        out += arr[i] + " & "
    
    out += arr[-1] + " " + end
    return out

In [8]:
set_col_name = "Sada"
resampling_col_name = "Převzorkovací období"
levels_unique_count_col_name = "Úrovně nákupu počet jedinečných hodnot"
levels_lower_bound_col_name = "Úrovně nákupu spodní hranice"
unique_sizes_count_col_name = "Velikosti nákupu počet jedinečných hodnot"
period_from_col_name = "Období od"
period_to_col_name = "Období do"
period_step_col_name = "Období krok"
types_col_name = "Indikátory"

set_overview = pd.DataFrame(columns = [
    set_col_name,
    resampling_col_name,
    levels_unique_count_col_name,
    levels_lower_bound_col_name,
    unique_sizes_count_col_name,
    period_from_col_name,
    period_to_col_name,
    period_step_col_name,
    types_col_name
])

In [9]:
for set_name in set_names:
    set_dir = os.path.join(optimizer_dir, set_name)
    experiment_dir = os.path.join(set_dir, "01")
    settings_path = os.path.join(experiment_dir, "settings.json")
    
    settings_file = open(settings_path, "r")
    settings_doc = json.loads(settings_file.read())
    
    resampling = settings_doc["resampling"]["period[min]"]
    
    search_space_doc = settings_doc["search space"]
    levels_doc = search_space_doc["levels"]
    levels_unique_count = levels_doc["unique count"]
    levels_lower_bound = levels_doc["lower bound"]
    unique_sizes_count = search_space_doc["open order sizes"]["unique count"]
    
    indicator_doc = search_space_doc["indicator"]
    period_doc = indicator_doc["period"]
    period_from = period_doc["from"]
    period_to = period_doc["to"]
    period_step = period_doc["step"]
    types = indicator_doc["types"]
    
    for i in range(len(types)):
        types[i] = types[i].upper()
        
    types = list_to_string(types)

    set_overview.loc[len(set_overview)] = {
        set_col_name : set_name.replace("set-", ""),
        resampling_col_name : resampling,
        levels_unique_count_col_name : levels_unique_count,
        levels_lower_bound_col_name : convert_level(levels_lower_bound),
        unique_sizes_count_col_name : unique_sizes_count,
        period_from_col_name : period_from,
        period_to_col_name : period_to,
        period_step_col_name : period_step,
        types_col_name : types,
    }

In [10]:
set_overview

,Sada,Převzorkovací období,Úrovně nákupu počet jedinečných hodnot,Úrovně nákupu spodní hranice,Velikosti nákupu počet jedinečných hodnot,Období od,Období do,Období krok,Indikátory
0,1,45,15,"-25,0",6,3,60,3,\([\begin{array}{cc} SMA & EMA \end{array}]\)
1,2,45,20,"-25,0",6,3,120,3,\([\begin{array}{c} SMA \end{array}]\)
2,3,15,15,"-40,0",11,3,105,3,\([\begin{array}{c} SMA \end{array}]\)


In [12]:
report_dir = "./data/report"
optimizer_dir = os.path.join(report_dir, optimizer_name)
set_overview.to_csv(os.path.join(optimizer_dir, "set-overview.csv"), index=False)